# ETL of DrugBank Approved Drugs Database

## Library importation

In [2]:
import pandas as pd
import sqlite3

## Dataframe

In [4]:
# Read csv file
df = pd.read_csv('approved.csv')

# Drop columns
df.drop(['CAS Number', 'Drug Groups', 'InChIKey','InChI','Formula','KEGG Compound ID', 
         'KEGG Drug ID', 'PubChem Substance ID', 'ChEBI ID', 'PubChem Compound ID', 'HET ID', 'ChemSpider ID', 'BindingDB ID', 'ChEMBL ID'], axis=1, inplace=True)

# Sort by ID
df = df.sort_values(by='DrugBank ID')

# Show dataframe
df.head()

,DrugBank ID,Name,SMILES
0,DB00006,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,DB00014,Goserelin,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
2,DB00027,Gramicidin D,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
3,DB00035,Desmopressin,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
4,DB00050,Cetrorelix,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...


### NaN and duplicated

In [5]:
# Duplicated rows
num_duplicadas = df.duplicated().sum()
print("Número de linhas duplicadas:", num_duplicadas)

# Show duplicated rows
duplicadas = df[df.duplicated()]
print("Linhas duplicadas:")
print(duplicadas)

# NaN values
nulos_por_coluna = df.isnull().sum()
print("Valores nulos por coluna:")
print(nulos_por_coluna)

Número de linhas duplicadas: 0
Linhas duplicadas:
Empty DataFrame
Columns: [DrugBank ID, Name, SMILES]
Index: []
Valores nulos por coluna:
DrugBank ID      0
Name             0
SMILES         157
dtype: int64


## Report
- There are no duplicate values ​​in the dataset

- DrugBank ID and Name columns do not have NaN values

- 157 compounds were identified that do not have values ​​in the SMILES column

## Separate Dataframes

In [6]:
# Create a separate DataFrame with the rows where the SMILES column is null
df_smiles_null = df[df['SMILES'].isnull()]
df_smiles_null.head()

,DrugBank ID,Name,SMILES
160,DB00286,Conjugated estrogens,NaN
246,DB00375,Colestipol,NaN
277,DB00407,Ardeparin,NaN
329,DB00460,Verteporfin,NaN
380,DB00516,Bentoquatam,NaN


In [7]:
# Remove rows with null values ​​in the SMILES column of the original DataFrame
df_clean = df.dropna(subset=['SMILES'])
df_clean.head()

,DrugBank ID,Name,SMILES
0,DB00006,Bivalirudin,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,DB00014,Goserelin,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
2,DB00027,Gramicidin D,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
3,DB00035,Desmopressin,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
4,DB00050,Cetrorelix,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...


In [ ]:
# Duplicated rows SMILES
df_clean['SMILES'].duplicated().sum()

np.int64(0)

In [9]:
# Save cleaned dataframe in a csv file
df_clean.to_csv('approved_drugs_db.csv', index=False)

In [10]:
# Save dataframe with null values in the SMILES column in a csv file
df_smiles_null.to_csv('smiles_null.csv', index=False)

## Save the SMILES in a csv file

In [11]:
df_smiles = df_clean['SMILES']
df_smiles.head()
df_smiles.to_csv('smiles.csv', index=False)

## New DataFrame

In [5]:
df = pd.read_csv('approved_drugs_db.csv', sep=';')
df.head()

,ID,Name,MW,SMILES
0,DB00006,Bivalirudin,2180.2853,CC[C@H](C)[C@H](NC(=O)[C@H](CCC(O)=O)NC(=O)[C@...
1,DB00014,Goserelin,1269.4105,CC(C)C[C@H](NC(=O)[C@@H](COC(C)(C)C)NC(=O)[C@H...
2,DB00027,Gramicidin D,1811.2530,CC(C)C[C@@H](NC(=O)CNC(=O)[C@@H](NC=O)C(C)C)C(...
3,DB00035,Desmopressin,1069.2200,NC(=O)CC[C@@H]1NC(=O)[C@H](CC2=CC=CC=C2)NC(=O)...
4,DB00050,Cetrorelix,1431.0380,CC(C)C[C@H](NC(=O)[C@@H](CCCNC(N)=O)NC(=O)[C@H...


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2648 entries, 0 to 2647
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      2648 non-null   object 
 1   Name    2648 non-null   object 
 2   MW      2648 non-null   float64
 3   SMILES  2648 non-null   object 
dtypes: float64(1), object(3)
memory usage: 82.9+ KB


In [7]:
df.isnull().sum()

ID        0
Name      0
MW        0
SMILES    0
dtype: int64

In [8]:
df.duplicated().sum()

np.int64(0)

In [9]:
df['SMILES'].duplicated().sum()

np.int64(0)

In [10]:
df['SMILES'].to_csv('SMILES.csv', index=False)

## Connect to a SQLite DB file

In [33]:
# Connect to database (will be created if it does not exist)
conn = sqlite3.connect('approved_drugs.db')

# Save the dataframe into a database table called Main
df.to_sql('Main', conn, if_exists='replace', index=False)

# Close connection
conn.close()